<a href="https://colab.research.google.com/github/stevenAnto/llm/blob/main/LangChaing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Ingresa tu clave de OpenAI: ")


Ingresa tu clave de OpenAI: ··········


In [2]:
!pip install langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.65
    Uninstalling langchain-core-0.3.65:
      Successfully uninstalled langchain-core-0.3.65


In [2]:
from langchain_openai import ChatOpenAI

In [3]:
llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])

In [4]:
llm.invoke("la profesion de IA tiene un futuro prometedor?")

AIMessage(content='Sí, la profesión de Inteligencia Artificial (IA) tiene un futuro muy prometedor. Con los avances tecnológicos cada vez más rápidos, la demanda de profesionales en IA seguirá creciendo en sectores como la salud, la banca, la industria, la logística, entre otros. Además, la IA tiene el potencial de revolucionar muchas industrias y mejorar la eficiencia y la productividad en diferentes ámbitos. Por lo tanto, aquellos que se formen y se especialicen en IA tienen grandes oportunidades laborales y de desarrollo profesional en el futuro.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 133, 'prompt_tokens': 19, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Blvm2fgSmKaaMQQb

In [5]:
from  langchain_core.prompts import ChatPromptTemplate

In [6]:
prompt = ChatPromptTemplate.from_messages([
    ("system","Eres un asistente de IA, puedes responder a preguntas sobre IA"),
    ("user","{input}")
])

In [7]:
chain  = prompt | llm
chain.invoke({"input":"la profesion de IA tiene un futuro prometedor?"})

AIMessage(content='Sí, la profesión de Inteligencia Artificial (IA) tiene un futuro muy prometedor. Con los avances tecnológicos en el campo de la IA, se espera que haya una creciente demanda de profesionales especializados en esta área. Las empresas de todo el mundo están adoptando la IA para mejorar sus operaciones, tomar decisiones más informadas y ofrecer productos y servicios más personalizados. Por lo tanto, si estás interesado en la IA, es un buen momento para considerar una carrera en este campo en constante evolución.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 39, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Blvm6Sd5cV3cD5p7uRmZtsAYaZx9l', 'servic

#Memory

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI


In [25]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Eres un asistente de {ability}, puedes responder a preguntas sobre {ability}",
     ),
    MessagesPlaceholder(variable_name="history"),#ingresa el historico de la conversacion
    ("human","{input}")
])

In [26]:
runnable = prompt | model

In [12]:
!pip install langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25


In [27]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [28]:
store = {}

In [29]:
#Para saber si hay contexto anterior
def get_session_history(session_id: str)->BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

In [32]:
with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_message_key="input",
    history_messages_key="history",
)

In [33]:
with_message_history.invoke(
    {"ability":"ciencia de datos","input":"que es una funcion de perdida"},
    config={"configurable":{"session_id":"abc123"}},
    )

AIMessage(content='Una función de pérdida, también conocida como función de costo, es una medida que se utiliza para evaluar qué tan bien está funcionando un modelo de machine learning en la predicción de valores de salida con respecto a los valores reales. Esta función calcula la diferencia entre la predicción del modelo y el valor real, generando un número que representa el error de la predicción.\n\nEl objetivo es minimizar esta función de pérdida durante el entrenamiento del modelo para que pueda realizar predicciones más precisas. Algunos ejemplos de funciones de pérdida comunes incluyen el error cuadrático medio (MSE), la entropía cruzada (para problemas de clasificación) y la función de bisagra (para problemas de clasificación de tipo SVM).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 167, 'prompt_tokens': 40, 'total_tokens': 207, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens'

In [34]:
with_message_history.invoke(
    {"ability":"ciencia de datos","input":"como?, no te entendi"},
    config={"configurable":{"session_id":"abc123"}},
    )

AIMessage(content='Claro, déjame explicarlo de una manera más sencilla. Una función de pérdida es una medida que nos dice qué tan bien está aprendiendo nuestro modelo de machine learning. Imagina que tienes un modelo que intenta predecir el precio de una casa. La función de pérdida calculará la diferencia entre el precio real de la casa y la predicción hecha por el modelo.\n\nEl objetivo es que esta diferencia sea lo más pequeña posible, ya que eso significa que nuestro modelo está haciendo predicciones más precisas. Durante el entrenamiento del modelo, ajustamos sus parámetros para minimizar esta función de pérdida, de modo que el modelo pueda aprender de los datos y hacer mejores predicciones. Hay diferentes tipos de funciones de pérdida, cada una adecuada para distintos tipos de problemas de machine learning.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 180, 'prompt_tokens': 221, 'total_tokens': 401, 'completion_tokens_details': {'ac

#Tools


In [37]:
!pip install wikipedia


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=93f09ae4e806ade9d4386568521238c0cd5818734ced7ea783f47ae4a792d34f
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [38]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [39]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [40]:
tool.name

'wikipedia'

In [41]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [42]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [51]:
tool.run({"query":"Alan Turing"})

'Page: Alan Turing\nSummary: Alan Mathison Turing (; 23 June 1912 – 7 June 1954) was an English mathematician, computer scientist, logician, cryptanalyst, philosopher and theoretical biologist. He was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer. Turing is widely considered to be the father of theoretical computer sc'

In [50]:
tool.run({"query":"quien es Alan Turing"})

'Page: Arturo Puig\nSummary: Arturo José Alberto Puig (born 17 November 1944) is an Argentine actor. He has worked in the 1990s success Grande, pá!!!'

#Agents


In [56]:
!pip install -U langchain-tavily


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 101.5 MB/s eta 0:00:00


In [53]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [62]:
api_key = getpass(" Ingresa tu Tavily API Key: ")
search = TavilySearchResults(tavily_api_key=api_key)

 Ingresa tu Tavily API Key: ··········


In [63]:
search.invoke("Que temperatura hay en Londres")

[{'title': 'El Clima en Londres 2025 | Londres 14 días | Tiempo3.com',
  'url': 'https://www.tiempo3.com/europe/united-kingdom/london',
  'content': 'No importa si eres ciudadano de Londres o turista, siempre es una buena idea estar al tanto del tiempo que va a hacer.\n\n[La temperatura actual en Londres](https://www.tiempo3.com/europe/united-kingdom/london?page=current-weather) es de °. La temperatura máxima en Londres para hoy será de ° y la mínima se encontrará en torno a los °.',
  'score': 0.84767824},
 {'title': 'Clima en Londres: la temperatura estación por estación - Assist Card',
  'url': 'https://www.assistcard.com/pe/clima-londres',
  'content': 'El verano londinense es templado, con temperaturas mínimas entre 13 y 16 ºC y máximas de 20 a 23 ºC. Los días soleados son frecuentes y las calles se llenan de',
  'score': 0.84215075},
 {'title': 'El clima y el tiempo promedio en todo el año en Londres Reino Unido',
  'url': 'https://es.weatherspark.com/y/45062/Clima-promedio-en-Lo

In [64]:
tools = [search, tool]

In [65]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0)

In [66]:
from langchain import hub

In [67]:
#prompt de la comunidad
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [68]:
from langchain.agents import create_tool_calling_agent

In [69]:
#Creacion de agentes
agent = create_tool_calling_agent(llm,tools,prompt=prompt)

In [70]:
from langchain.agents import AgentExecutor

In [71]:
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [72]:
agent_executor.invoke({"input":"hola"})



> Entering new AgentExecutor chain...
¡Hola! ¿En qué puedo ayudarte hoy?

> Finished chain.


{'input': 'hola', 'output': '¡Hola! ¿En qué puedo ayudarte hoy?'}

In [74]:
agent_executor.invoke({"input":"cual es el mejor  superheroe de Marvel y por que?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'List of Marvel Comics characters'}`


Page: Lists of Marvel Comics characters
Summary: This is a list of Marvel multiverse fictional characters which were created for and are owned by Marvel Comics. Licensed or creator-owned characters (G.I. Joe, Godzilla, Groo the Wanderer, Men in Black, Conan the Barbarian,  Mighty Morphin Power Rangers, RoboCop, Star Trek, Rocko's Modern Life, The Ren and Stimpy Show, etc.) are not included.
Marvel-Electronic Arts video game characters are also included with references. Characters from the MarvelLa pregunta sobre quién es el mejor superhéroe de Marvel es subjetiva y puede variar según las preferencias de cada persona. Marvel Comics tiene una amplia gama de personajes icónicos y populares, como Spider-Man, Iron Man, Captain America, Thor, Hulk, Black Widow, entre muchos otros. Cada uno de estos superhéroes tiene sus propias historias, habilidades y seguidores que los cons

{'input': 'cual es el mejor  superheroe de Marvel y por que?',
 'output': 'La pregunta sobre quién es el mejor superhéroe de Marvel es subjetiva y puede variar según las preferencias de cada persona. Marvel Comics tiene una amplia gama de personajes icónicos y populares, como Spider-Man, Iron Man, Captain America, Thor, Hulk, Black Widow, entre muchos otros. Cada uno de estos superhéroes tiene sus propias historias, habilidades y seguidores que los consideran los mejores por diferentes razones.\n\nSi deseas más información sobre los superhéroes de Marvel para poder elegir cuál consideras el mejor, puedo buscar una lista detallada de personajes de Marvel Comics. ¿Te gustaría que busque más información al respecto?'}

In [75]:
agent_executor.invoke({"input":"Que temperatura hace en Nueva York?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'temperatura en Nueva York'}`


[{'title': 'Tiempo para hoy en Nueva York (Local): Hora por hora - Tiempo3', 'url': 'https://www.tiempo3.com/north-america/usa/new-york/new-york-city?page=today', 'content': 'Teniendo en cuenta el viento, la humedad, y otras condiciones climáticas, la sensación térmica será de 30°C.\n\nLa probabilidad de lluvias será del 0% y la velocidad del viento será de 8 Km/h.\n\n![Soleado]( "Soleado")\n\nPara el mediodía de hoy se espera una temperatura de 38°C, y una sensación térmica de 46°C en Nueva York.\n\nLa humedad del ambiente será de 45% y el viento soplará con una velocidad de 13 Km/h.\n\n![Soleado]( "Soleado")\n\nPor la tarde, la temperatura en Nueva York será de 40°C. [...] La probabilidad de lluvias será del 0% y el viento tendrá una velocidad de 14 Km/h.\n\n![Despejado]( "Despejado")\n\nHoy por la noche, en Nueva York, se espera una temperatura de 33°C y un

{'input': 'Que temperatura hace en Nueva York?',
 'output': 'La temperatura actual en Nueva York es de 98°F (aproximadamente 37°C). Puedes obtener más detalles sobre el clima actual en [AccuWeather](https://www.accuweather.com/es/us/new-york/10021/current-weather/349727).'}